In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


## 3950 Assignment 1: Part 2

In [2]:
name = "Michael Harris"

#### Data Setup

In [3]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
162,1,0.814,0.195,0.511,0.121,0.804,0.650,0.114,0.654,0.428,...,0.405,0.908,0.786,0.430,0.350,0.974,0.134,0.002,0.039,0.940
224,1,0.729,0.445,0.775,0.286,0.441,0.345,0.216,0.790,0.697,...,0.077,0.363,0.005,0.018,0.850,0.682,0.390,0.086,0.223,0.077
71,0,0.295,0.129,0.070,0.859,0.849,0.248,0.629,0.690,0.084,...,0.488,0.911,0.526,0.466,0.979,0.378,0.551,0.356,0.659,0.532
119,0,0.679,0.345,0.321,0.411,0.279,0.700,0.046,0.117,0.762,...,0.666,0.306,0.552,0.754,0.013,0.581,0.030,0.662,0.516,0.641
19,0,0.088,0.813,0.837,0.104,0.478,0.264,0.868,0.110,0.406,...,0.930,0.535,0.799,0.207,0.074,0.138,0.138,1.000,0.516,0.599


In [4]:
# Create testing and training data
df2 = pd.get_dummies(df, drop_first=True)
y = np.array(df2["target"]).reshape(-1,1)
X = np.array(df2.drop(columns={"target"}))
X_train, X_test, y_train, y_test = train_test_split(X, y)

#### Modelling

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Create model using standard scalar and a forest in a pipeline
scaler = StandardScaler()
estimator = RandomForestClassifier(n_jobs=-1)
pipe = Pipeline(steps=[("scaler", scaler), ("forest", estimator)])

params = {'forest__max_depth':[6,7,8,9],
          'forest__min_samples_split':[3,4,5,6,7],
          'forest__min_samples_leaf' :[2,3,4,5,6],
          'forest__n_estimators' :[75,80,85],
          'forest__criterion':["entropy"]}
 
best = GridSearchCV(pipe, param_grid=params, cv=5, n_jobs=-1) 
best.fit(X_train, y_train.ravel())
best1 = best.best_estimator_
print(best1.score(X_test, y_test))
print(best1)

0.5714285714285714
Pipeline(steps=[('scaler', StandardScaler()),
                ('forest',
                 RandomForestClassifier(criterion='entropy', max_depth=9,
                                        min_samples_leaf=6, min_samples_split=5,
                                        n_estimators=85, n_jobs=-1))])


### Finishing

In [22]:
print(best.score(X_test, y_test))
print(best)

0.6031746031746031
GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('forest',
                                        RandomForestClassifier(n_jobs=-1))]),
             n_jobs=-1,
             param_grid={'forest__criterion': ['entropy'],
                         'forest__max_depth': [6, 7, 8, 9],
                         'forest__min_samples_leaf': [2, 3, 4, 5, 6],
                         'forest__min_samples_split': [3, 4, 5, 6, 7],
                         'forest__n_estimators': [75, 80, 85]})


### Testing


In [29]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


0.6396708010582511
0.6394430379746835
Michael Harris 0.6395569195164673


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<ul>
<li> For the model I used a forest classifier instead of a single tree, in combination with a standard scalar in a pipeline.
<li> For that forest model I used a grid search with a combination of four parameters: max depth, minimum sample split, minimum sample leaf and n estimators. I also set the criterion to entropy.
</ul>